In [ ]:
import os
import numpy as np
import rasterio
import shutil
from vis_tools import read_hysplit_netcdf

FILENAME = '18042918_taupo_15.0_0.01.nc'

folder = FILENAME.rstrip('.nc')
if not os.path.exists(folder):
    os.makedirs(folder)   
    
# read netCDF (remove the trivial, empty t=0 step and transpose for GeoTIFF coord order)
temp = read_hysplit_netcdf(FILENAME).isel(time=slice(1, None)).transpose('time', 'lat', 'lon')

nodata = -47
np.place(temp['total_deposition'].values, np.isnan(temp['total_deposition'].values), nodata)  # transform all NaN's to nodata

temp.to_netcdf('temp.nc')
model = rasterio.open('temp.nc', 'r')
os.remove('temp.nc')

for time_step in model.indexes:
    
    timestamp = str(np.datetime64(temp['time'].values[time_step-1], 'h'))
    
    fname = FILENAME.rstrip('.nc')+'_'+timestamp+'.tif'
    raster = rasterio.open(os.path.join(folder, fname),
                           'w', driver='GTiff',
                           height=model.height, width=model.width,
                           count=1, dtype=model.profile['dtype'],
                           crs={'init': 'epsg:4326'}, transform=model.affine,
                           nodata=nodata)
    raster.write(model.read(time_step), 1)  
    shared_profile = raster.profile
    raster.close()
    print('finished writing {}'.format(fname))
    
model.close()

shutil.make_archive(folder, 'zip', folder)

print('done')

In [ ]:
import fiona
from fiona.crs import from_epsg
from vis_tools import read_hysplit_netcdf
from shapely.geometry import Point, mapping
import os
import shutil

FILENAME = '18042918_taupo_15.0_0.01.nc'

folder = 'taupo'
if not os.path.exists(folder):
    os.makedirs(folder)  

model = read_hysplit_netcdf(FILENAME)

schema =  {'geometry': 'Point',
           'properties': {'volc_name': 'str'}}

with fiona.open(folder+'/deleteme.shp', 'w', crs=from_epsg(4326), driver='ESRI Shapefile', schema=schema) as output:    
    # geometry
    point = mapping(Point(model.attrs['volcano_location'][1], model.attrs['volcano_location'][0]))

    # write the row (geometry + attributes in GeoJSON format)
    output.write({'geometry': point, 'properties': {'volc_name': 'Taupo'}})
            
shutil.make_archive('taupo', 'zip', 'taupo')  

shutil.rmtree(folder)

In [ ]:
import numpy as np
import simplekml
from matplotlib.cm import ScalarMappable
from matplotlib.colors import LogNorm
import colorcet as cc
from vis_tools import read_hysplit_netcdf
from pandas import Timestamp
from pytz import timezone
NZT = timezone('Pacific/Auckland')

import time
t_i = time.time()

FILENAME = '18042918_taupo_15.0_0.01.nc'

ASH_MIN = 10**-1
ASH_MAX = 10**2

model = read_hysplit_netcdf(FILENAME)

cell_dims = np.unique(np.concatenate((np.ediff1d(model['lon']), np.ediff1d(model['lat']))))
print('unique cell dimensions found: {}'.format(cell_dims))
cell_size = np.mean(cell_dims)
print('cell size interpretation: {}\n'.format(cell_size))

model_bounds = dict(
                    left=model['lon'].values[0] - cell_size/2,
                    right=model['lon'].values[-1] + cell_size/2,
                    bottom=model['lat'].values[0] - cell_size/2,
                    top=model['lat'].values[-1] + cell_size/2
                   )
cell_bounds = dict(
                   x=np.linspace(model_bounds['left'], model_bounds['right'], len(model['lon'])+1),
                   y=np.linspace(model_bounds['bottom'], model_bounds['top'], len(model['lat'])+1)
                  )

CMAP = cc.m_linear_kry_5_95_c72_r
mapper = ScalarMappable(norm=LogNorm(vmin=ASH_MIN, vmax=ASH_MAX), cmap=CMAP)
cmap_rgba = CMAP(np.arange(CMAP.N), bytes=True)
cmap_kml_hex = [simplekml.Color.rgb(*rgba) for rgba in cmap_rgba]     

stylemaps = []
for kml_hex in cmap_kml_hex:
    
    color_with_alpha = simplekml.Color.changealphaint(127, kml_hex) 
    
    sm = simplekml.StyleMap()
    
    sm.normalstyle.polystyle.outline = 0
    sm.normalstyle.iconstyle.scale = 0
    sm.normalstyle.iconstyle.icon = None
    sm.normalstyle.iconstyle.heading = None
    sm.normalstyle.iconstyle.colormode = None
    sm.normalstyle.labelstyle.scale = 0
    sm.normalstyle.labelstyle.colormode = None
    sm.normalstyle.polystyle.color = color_with_alpha
    sm.normalstyle.polystyle.colormode = None
     
    sm.highlightstyle.polystyle.outline = None  # this ENABLES the outline
    sm.highlightstyle.iconstyle.scale = 0
    sm.highlightstyle.iconstyle.icon = None
    sm.highlightstyle.iconstyle.heading = None
    sm.highlightstyle.iconstyle.colormode = None
    sm.highlightstyle.labelstyle.scale = 1  # make label appear when cell is hovered over
    sm.highlightstyle.labelstyle.colormode = None
    sm.highlightstyle.polystyle.color = color_with_alpha
    sm.highlightstyle.polystyle.colormode = None

    sm.highlightstyle.linestyle.width = 5  # make an outline appear when cell is hovered over
    sm.highlightstyle.linestyle.colormode = None
        
    stylemaps.append(sm)

shared_styles = dict(zip(cmap_kml_hex, stylemaps))  

kml = simplekml.Kml()

for time_step in model['time'].values:
        
    fol = kml.newfolder(name=str(np.datetime64(time_step, 'h')))
    
    fol.timespan.begin = NZT.localize(Timestamp(time_step).to_pydatetime()).isoformat()
    ts = np.mean(np.ediff1d(model['time']).astype('timedelta64[h]'))
    fol.timespan.end = NZT.localize(Timestamp(time_step+ts).to_pydatetime()).isoformat()
    
    fol.style.liststyle.listitemtype = 'checkHideChildren'
    
    for x_ind in range(len(model['lon'])):#range(400, 450):
        for y_ind in range(len(model['lat'])):#range(225, 275):        
            cell = {}

            value = model.sel(time=time_step)['total_deposition'].values[y_ind, x_ind]

            if np.isnan(value):
                pass
            else:        
                cell['left'] = cell_bounds['x'][x_ind]
                cell['right'] = cell_bounds['x'][x_ind+1]        
                cell['bottom'] = cell_bounds['y'][y_ind]
                cell['top'] = cell_bounds['y'][y_ind+1]
                cell['x'] = model['lon'].values[x_ind]        
                cell['y'] = model['lat'].values[y_ind]        

                geo = fol.newmultigeometry(name='{:.3g} mm'.format(value))

                pnt = geo.newpoint(altitudemode='absolute')
                poly = geo.newpolygon(altitudemode='clampToGround') 

                pnt.coords = [(cell['x'], cell['y'])]
                poly.outerboundaryis = [
                                        (cell['left'], cell['bottom']),
                                        (cell['left'], cell['top']),
                                        (cell['right'], cell['top']),
                                        (cell['right'], cell['bottom']),
                                        (cell['left'], cell['bottom'])
                                       ]

                rgba = mapper.to_rgba(value, bytes=True)
                kml_hex = simplekml.Color.rgb(*rgba)
                geo.stylemap = shared_styles[kml_hex]
                
                del value
                del pnt
                del poly
                        
    print('{} written'.format(time_step))
    
kml.save(FILENAME.rstrip('.nc')+'.kml')
print('KML file written\n')

t_f = time.time()
print('{:.0f} minute(s) elapsed'.format((t_f - t_i)/60))